# Création du modèle d'estimation des prix

### Import des modules

In [ ]:
import pandas as pd
import os
import datetime
import seaborn as sns

from tqdm import tqdm

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor


from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import tensorflow as tf

from scipy.sparse import hstack


## Préparation des données

In [18]:
df_text = pd.read_csv(os.path.join('../..','data/4/raw_data/full_scrap_clean_features_only.csv'))
df_img = pd.read_csv(os.path.join('../..','data/4/raw_data/full_scrap_clean_images_only.csv'))

In [19]:
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6427 entries, 0 to 6426
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        6427 non-null   object
 1   image_1   6278 non-null   object
 2   image_2   5001 non-null   object
 3   image_3   4365 non-null   object
 4   image_4   3285 non-null   object
 5   image_5   2488 non-null   object
 6   image_6   1804 non-null   object
 7   image_7   1314 non-null   object
 8   image_8   961 non-null    object
 9   image_9   703 non-null    object
 10  image_10  402 non-null    object
 11  image_11  2 non-null      object
dtypes: object(12)
memory usage: 602.7+ KB


In [20]:
df_img = df_img.loc[:,df_img.isnull().mean()<0.5]

In [21]:
df_imgtxt = df_text.merge(df_img,how='left',on='id')
df_imgtxt['price']= df_imgtxt['price'].str.replace('€','').str.replace('.','').astype(float)
df_imgtxt = df_imgtxt.drop(columns='other_features')

In [22]:
df_imgtxt = df_imgtxt[(df_imgtxt['price']>100) & (df_imgtxt['price']< 8000)]

In [24]:
df_imgtxt['Nombres de vitesses'] = df_imgtxt['Nombres de vitesses'].str.extract(r'(\d+)').astype(float)

In [25]:
df_imgtxt['title_desc'] = (df_imgtxt['title'].fillna('') + "_" + df_imgtxt['description'].fillna(''))
df_imgtxt.drop(columns=['title','description'], inplace=True)
df_imgtxt=df_imgtxt[df_imgtxt['price']<10000]

In [26]:
df_imgtxt.drop(columns=['Couleurs principales'], inplace=True)

In [27]:
df_imgtxt['Année'] = datetime.date.today().year - df_text['Année']

In [28]:
size_mapping = {
    'XXS': 'XS',
    'XS': 'XS',
    'S': 'S',
    'S/M': 'S',
    'M': 'M',
    'M/L': 'M',
    'L': 'L',
    'L/XL': 'L',
    'XL': 'XL',
    'XXL': 'XXL',

    # Valeurs en cm
    '44 cm': 'XS',
    '45 cm': 'XS',
    '46 cm': 'XS',
    '47 cm': 'XS',
    '48 cm': 'XS',
    '49 cm': 'S',
    '50 cm': 'S',
    '51 cm': 'S',
    '52 cm': 'S',
    '53 cm': 'S',
    '54 cm': 'M',
    '55 cm': 'M',
    '56 cm': 'M',
    '57 cm': 'L',
    '58 cm': 'L',
    '59 cm': 'L',
    '60 cm': 'XL',
    '61 cm': 'XL',
    '62 cm': 'XL',
    '63 cm': 'XXL',
    '64 cm': 'XXL',

    # Autres cas
    'Unique': 'M',
    'EXPERT': 'XL',
    'PRO': 'XL',
    'PRO XL': 'XL',
    'JUNIOR': 'XS'
}
df_imgtxt['Taille du cadre'] = df_imgtxt['Taille du cadre'].map(size_mapping)

In [30]:
df_imgtxt['Taille du cadre'].value_counts()

Taille du cadre
M      2750
S      1516
L       979
XS      379
XL      226
XXL      19
Name: count, dtype: int64

In [31]:
df_imgtxt.loc[~(df_imgtxt['Diametre roues'] == '700 / 28"'), 'Diametre roues'] = 'Autre'

In [32]:
df_imgtxt['Diametre roues'].value_counts()

Diametre roues
700 / 28"    4698
Autre        1528
Name: count, dtype: int64

## Définition du modèle

In [34]:
cnn_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

## Transformation des images en vecteurs

In [ ]:
def extract_image_features(img_path, model):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = preprocess_input(np.expand_dims(x, axis=0))
        features = model.predict(x, verbose=0)
        return features.flatten()
    except Exception as e:
        print(f"Erreur pour {img_path}: {e}")
        return np.full((1280,), np.nan)


In [38]:
example_path = df_imgtxt['image_1'].iloc[0]
print(example_path)
features_vector = extract_image_features(example_path, cnn_model)
print("Forme du vecteur :", features_vector.shape)
print("Premières valeurs :", features_vector[:10])


/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00000_0.jpg
Forme du vecteur : (1280,)
Premières valeurs : [ 0.04356423 -0.04198746 -0.13860649  0.32269746 -0.1366466   0.08849818
 -0.08942988 -0.02371817 -0.11457141 -0.07759564]


In [42]:
tqdm.pandas()  # pour ajouter .progress_apply
df_imgtxt['img_features'] = df_imgtxt['image_1'].progress_apply(lambda path: extract_image_features(path, cnn_model))

  0%|          | 10/6226 [00:00<03:45, 27.59it/s]

Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00004_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00004_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00005_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00005_0.jpg'
Erreur pour nan: path should be path-like or io.BytesIO, not <class 'float'>


  0%|          | 16/6226 [00:00<04:05, 25.32it/s]

Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00012_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00012_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00014_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_00014_0.jpg'


  0%|          | 22/6226 [00:00<04:32, 22.78it/s]

...

Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06410_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06410_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06412_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06412_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06414_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06414_0.jpg'


100%|█████████▉| 6221/6226 [03:37<00:00, 25.31it/s]

Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06417_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06417_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06418_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06418_0.jpg'


100%|██████████| 6226/6226 [03:37<00:00, 28.58it/s]

Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06422_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06422_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06423_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06423_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06424_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06424_0.jpg'
Erreur pour /Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06426_0.jpg: [Errno 2] No such file or directory: '/Users/vloi/code/vloi/helperz/data_viz/data/4/img_tvlo_2/tvlo_06426_0.jpg'


In [43]:
df_imgtxt = df_imgtxt[df_imgtxt['img_features'].apply(lambda x: isinstance(x, np.ndarray) and not np.isnan(x).any())].reset_index(drop=True)

In [49]:
df_img_features = pd.DataFrame(df_imgtxt['img_features'].tolist(), index=df_imgtxt.index)
df_img_features.columns = [f'img_feat_{i}' for i in range(df_img_features.shape[1])]
df_img_features

,img_feat_0,img_feat_1,img_feat_2,img_feat_3,img_feat_4,img_feat_5,img_feat_6,img_feat_7,img_feat_8,img_feat_9,...,img_feat_1270,img_feat_1271,img_feat_1272,img_feat_1273,img_feat_1274,img_feat_1275,img_feat_1276,img_feat_1277,img_feat_1278,img_feat_1279
0,0.043564,-0.041987,-0.138606,0.322697,-0.136647,0.088498,-0.089430,-0.023718,-0.114571,-0.077596,...,0.206662,0.367479,-0.121593,0.225629,-0.168089,-0.055242,-0.145003,-0.034050,-0.122637,0.086692
1,-0.056577,-0.146264,-0.130088,0.160847,0.170141,-0.050856,0.061493,0.295647,0.348297,0.022711,...,0.071999,0.237079,0.468186,0.367336,-0.144637,0.129026,-0.048032,0.040221,-0.146072,0.284620
2,-0.011964,-0.036959,-0.135223,0.444666,-0.152459,0.173199,-0.189654,-0.084439,-0.151969,-0.122831,...,-0.108195,0.006782,-0.125246,1.369407,-0.093290,-0.134700,-0.124489,-0.150111,-0.120382,-0.080500
3,-0.087820,0.331095,-0.178482,0.565397,-0.089394,-0.028082,-0.140699,-0.104097,0.116620,-0.077723,...,0.013915,0.616243,-0.169596,0.182080,-0.068169,0.191764,-0.115486,-0.143424,-0.147046,-0.167195
4,0.035376,0.178977,-0.180860,0.639732,-0.155814,-0.056426,-0.107038,-0.152742,-0.112681,-0.147236,...,-0.120137,0.341330,-0.039910,0.338668,-0.182072,-0.029636,-0.137257,-0.128793,-0.146748,-0.130013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3363,-0.190030,0.146327,-0.160143,0.090262,-0.122726,0.273766,-0.186499,-0.132644,-0.014611,0.015651,...,-0.107655,0.651925,-0.103259,0.161327,-0.193366,0.180467,-0.096692,-0.102837,-0.167618,-0.170053
3364,0.030806,-0.157874,-0.073826,0.074540,-0.146236,-0.174533,-0.063352,-0.149028,0.194552,-0.078873,...,-0.067453,-0.137237,-0.120400,0.022885,-0.154394,1.712546,-0.102164,-0.074191,-0.159493,-0.147748
3365,-0.038700,-0.151122,-0.152318,0.178465,-0.146924,0.007554,-0.124384,-0.045378,0.063907,0.013180,...,-0.107258,-0.004641,-0.140127,0.065862,-0.048648,-0.021418,-0.061439,-0.127009,-0.102244,0.277034
3366,-0.071478,0.225905,-0.141796,0.371283,-0.077385,-0.016519,-0.148058,-0.148209,-0.116357,0.026227,...,-0.135594,0.134193,-0.105650,0.789211,-0.111317,-0.111456,-0.110806,-0.121226,-0.155109,-0.097128


In [51]:
df_imgtxt.head(10)

,id,price,Année,État,Groupe,Taille du cadre,Diametre roues,Matière principale,Type de freinage,Nombres de vitesses,"Groupe électrique (AXS, Di2, eTap, ..)",Pratique,image_1,image_2,image_3,image_4,title_desc,img_features
0,i_00000,4200.0,4.0,Très bon état,Shimano Ultegra Di2,L,"700 / 28""",Carbone,Disques,11.0,Oui,NaN,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,NaN,NaN,NaN,Colnago V3rs - COLNAGO - 63000_État nickel (pr...,"[0.043564226, -0.041987464, -0.13860649, 0.322..."
1,i_00001,6700.0,0.0,Neuf,Shimano Dura Ace Di2,L,"700 / 28""",Carbone,Disques,12.0,NaN,Route,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,LOOK 795 BLADE RS - LOOK - 67720_Vends : LOOK...,"[-0.056576755, -0.14626409, -0.1300877, 0.1608..."
2,i_00002,250.0,45.0,Très bon état,NaN,L,"700 / 28""",Acier,NaN,NaN,NaN,Piste,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,NaN,NaN,NaN,Vélo de piste Lejeune vintage - LEJEUNE - 7895...,"[-0.011963621, -0.03695884, -0.13522252, 0.444..."
3,i_00006,1100.0,2.0,Très bon état,Shimano Tiagra,S,Autre,Aluminium,Patins,10.0,Non,NaN,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,Vektor Rigel - VEKTOR - 59320_Vente de mon Vek...,"[-0.08782002, 0.33109528, -0.17848235, 0.56539..."
4,i_00007,4490.0,5.0,Très bon état,SRAM Force AXS,S,"700 / 28""",Carbone,Disques,12.0,Oui,Route,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,Vélo de route Trek Madone SLR Sram Force AXS T...,"[0.03537588, 0.17897688, -0.1808602, 0.6397317..."
5,i_00009,3990.0,0.0,Comme neuf,Shimano Dura Ace Di2,S,Autre,Carbone,Disques,NaN,NaN,Route,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,Cadre specialized s-works tarmac sl8 2025 - AU...,"[-0.13109401, -0.08917615, 0.07930783, 0.01827..."
6,i_00010,1900.0,4.0,Très bon état,Shimano Ultegra,M,Autre,Carbone,Disques,NaN,Non,Route,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,NaN,NaN,BMC Roadmachine - AUTRE - 71570_BMC Roadmachin...,"[-0.10470385, 0.02714625, -0.1593556, 0.706642..."
7,i_00011,2800.0,1.0,Comme neuf,Shimano Ultegra Di2,M,"700 / 28""",Carbone,Disques,12.0,Oui,Route,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,NaN,NaN,NaN,Lapierre Xelius 7.0 - LAPIERRE - 25000_Vends v...,"[-0.1462329, 0.3726971, -0.13299076, 0.3521990..."
8,i_00013,1600.0,7.0,Comme neuf,Shimano Ultegra Di2,S,"700 / 28""",Carbone,Patins,11.0,Oui,Route,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,Giant TCR advanced SL Team Alpecin - GIANT - ...,"[-0.11813309, -0.039620776, -0.109453656, 0.32..."
9,i_00015,700.0,8.0,Bon état,SRAM Force,M,Autre,Aluminium,Patins,NaN,NaN,Route,/Users/vloi/code/vloi/helperz/data_viz/data/4/...,NaN,NaN,NaN,Cannondale caad 12 t56 - CANNONDALE - 29217_Ca...,"[0.19784217, -0.17573164, 0.32100636, 1.267573..."


In [74]:
X_tab = df_imgtxt.drop(columns=['price', 'id', 'image_1', 'img_features'])
y = df_imgtxt['price']

## 🔭 Preprocessor

### Définition des colonnes

In [60]:
cat_cols_onehot = [
    'Groupe',
    'Taille du cadre',
    'Diametre roues',
    'Matière principale',
    'Type de freinage',
    'Pratique',
    'Groupe électrique (AXS, Di2, eTap, ..)']
cat_cols_ordinal = ['État']
num_cols = ['Année',
            'Nombres de vitesses']
bool_cols = ['Groupe électrique (AXS, Di2, eTap, ..)']

### Définition du preprocessor

In [64]:
cat_pipeline_onehot = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
cat_pipeline_ordinal = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder(
    categories=[['Usé', 'Bon état', 'Très bon état','Comme neuf', 'Neuf']],
    handle_unknown='use_encoded_value',
    unknown_value=-1))
])
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('encoder', RobustScaler())
])
boolean_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Non')),
    ('encoder', OrdinalEncoder(categories=[['Non','Oui']],handle_unknown='use_encoded_value',unknown_value=-1))
])
boolean_pipeline

Pipeline(steps=[('imputer',
                 SimpleImputer(fill_value='Non', strategy='constant')),
                ('encoder',
                 OrdinalEncoder(categories=[['Non', 'Oui']],
                                handle_unknown='use_encoded_value',
                                unknown_value=-1))])

In [65]:
preprocessor = ColumnTransformer(transformers=[
    ('cat_oo', cat_pipeline_onehot, cat_cols_onehot),
    ('cat_or',cat_pipeline_ordinal, cat_cols_ordinal),
    ('num', num_pipeline, num_cols),
    ('text', TfidfVectorizer(max_features=1000),'title_desc'),
    ('bool', boolean_pipeline, bool_cols)
], remainder='drop')
preprocessor

ColumnTransformer(transformers=[('cat_oo',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Groupe', 'Taille du cadre', 'Diametre roues',
                                  'Matière principale', 'Type de freinage',
                                  'Pratique',
                                  'Groupe électrique (AXS, Di2, eTap, ..)']),
                                ('cat_or',
                                 Pipeline(steps=[('imputer',
                                                  Simple...
                                                 ('encoder', RobustScaler())]),
                                 ['Année', 'Nombres de vitesses']),
                                ('text', TfidfVectorizer(max_features=1000),
                                 'title_desc'),
                                ('bool',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Non',
                                                                strategy='constant')),
                                                 ('encoder',
                                                  OrdinalEncoder(categories=[['Non',
                                                                              'Oui']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1))]),
                                 ['Groupe électrique (AXS, Di2, eTap, ..)'])])

## 🏙️ Jointure des features text et image

In [68]:
X_text_processed = preprocessor.fit_transform(X_tab)

In [ ]:
X_image_array = df_img_features.values  # numpy array (n_samples, 1280)
if hasattr(X_text_processed,"todense"):
    X_combined = hstack([X_text_processed, X_image_array])
else :
    X_combined = np.hstack([X_text_processed, X_image_array])

## 🗿 Modèle

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [80]:
model = Ridge()
model.fit(X_train, y_train)

Ridge()

In [82]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2']
}
gridsearch_rf = GridSearchCV(
    rf,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1,
    n_jobs=-1
)
gridsearch_rf.fit(X_train, y_train)

best_rf_model = gridsearch_rf.best_estimator_
y_pred = best_rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print("✅ Meilleurs paramètres :", gridsearch_rf.best_params_)
print("🎯 MAE modèle RF multimodal :", round(mae, 2))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
✅ Meilleurs paramètres : {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100}
🎯 MAE modèle RF multimodal : 824.27


Victor, ta réflexion est **extrêmement pertinente**. Tu es en train de mettre le doigt sur les **vraies limites de la fusion naïve de features**, et tu poses des questions très justes. On prend ton raisonnement **point par point**, en gardant la rigueur mathématique que tu veux.

---

## 🔷 1. **Est-ce que toutes les features ont le même impact ?**

> Oui… **dans la manière dont on a fusionné les features**, **toutes les dimensions** (texte + image) sont mises côte à côte **sans distinction de source, d'échelle, ni de sens**.

👉 C’est ce qu’on appelle une **concaténation brute** :
Tu donnes 2000 colonnes au modèle, et il se débrouille. Le problème ?

* Le modèle **ne sait pas** que certaines colonnes viennent du texte (fort signal métier),
* Et que d'autres sont des projections visuelles très abstraites.

🧠 Résultat : tu dilues ton information utile dans du bruit → baisse des performances.

---

## 🔷 2. **Est-ce qu’une PCA sur l’image a du sens ici ?**

Tu as raison de douter, et ta justification est **pile dans le mille** :

> Les images sont toutes très différentes : angles, fond, zoom, etc. Donc faire une PCA sur les 1280 dimensions revient à chercher une base globale qui résume “toutes les images”. Or les motifs pertinents (ex : freins, cadre, selle) **n'apparaissent jamais au même endroit**, donc la variance globale ne correspond pas forcément au **sens métier**.

✅ PCA **peut aider à réduire le bruit**, mais **risque d’écraser des signaux faibles mais précieux**.
👉 On peut quand même la tester pour vérifier si elle améliore l’intégration dans le modèle (hypothèse à tester).

---

## 🔷 3. **Et si on pondérait moins fort les features d’image ?**

Très bonne piste. En concaténation brute :

```python
X_combined = np.hstack([X_text, X_image])
```

Chaque colonne a **le même poids initial**.

### 🧠 Solution possible :

* **Multiplie manuellement les features image par un facteur < 1**, par exemple 0.3

```python
X_combined = np.hstack([
    X_text,
    0.3 * X_image
])
```

Tu forces le modèle à :

* **Favoriser les features texte** (qui ont plus de signal métier)
* Réduire le **poids des features images** (sauf si elles sont vraiment utiles)

➡️ Ça revient à une **régularisation asymétrique**, et c’est très courant dans les architectures profondes multimodales (on fait ça implicitement avec des poids appris).

---

## 🔷 4. Et maintenant ?

Tu as plusieurs pistes simples à tester :

### ✅ Option A : pondérer les features image

```python
X_combined = np.hstack([X_text_dense, 0.3 * X_image_array])
```

Puis refaire le `train_test_split` + GridSearch avec RandomForest ou Ridge.

### ✅ Option B : tester les images **seules**

Entraîne un modèle **juste sur les vecteurs d’image**, pour quantifier leur valeur propre :

```python
X_img = X_image_array
y_img = df_imgtxt['price']
```

➡️ Si tu trouves une MAE > 1000 €, alors l’image seule **n’a pas de valeur prédictive forte**.

---

Souhaites-tu qu’on commence par **tester l’option A** (réduction du poids des features image) pour voir si ça améliore ton score ?
